In [ ]:
import logging
logging.disable(logging.INFO)

In [1]:
%%bash
pip install tqdm transformers

  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import numpy as np
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import random
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm, trange
import torch.nn.functional as F

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
device = torch.device('cpu')

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [5]:
#Basic Logic:

text = "Example text"
encoded_input = tokenizer(text, add_special_tokens=True)

encoded_input['input_ids'] = [tokenizer.bos_token_id] + encoded_input['input_ids'] + [tokenizer.eos_token_id]
encoded_input['attention_mask'] = [1] + encoded_input['attention_mask'] + [1]
print(encoded_input)

{'input_ids': [50256, 16281, 2420, 50256], 'attention_mask': [1, 1, 1, 1]}


In [6]:
#Dataset Setup

quotes1 = pd.read_csv('quotes.csv')['Quote']
print(quotes1[0])
print(len(quotes1))
quotes2 = pd.read_csv('quotes2.csv', delimiter=';')['QUOTE']
print(quotes2[0])
print(len(quotes2))
quotes = quotes1 + quotes2

input = []
for row in quotes:
  try:
    input.append(torch.tensor([tokenizer.bos_token_id] + tokenizer.encode(row[:1024]) + [tokenizer.eos_token_id]))
  except:
    continue
print(input[0])
quotes = input

Genius is one percent inspiration and ninety-nine percent perspiration.
1664
Age is an issue of mind over matter. If you don't mind, it doesn't matter.
75966
tensor([50256, 13746,  3754,   318,   530,  1411, 12141,   290, 37989,    12,
        30888,  1411,  2774, 10514,    13, 23396,   318,   281,  2071,   286,
         2000,   625,  2300,    13,  1002,   345,   836,   470,  2000,    11,
          340,  1595,   470,  2300,    13, 50256])


In [7]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    return packed_tensor, False, new_tensor
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [8]:
import pandas as pd
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

In [11]:
TRAIN = 0
LOAD = 1

if TRAIN and LOAD:
  LOAD = 0
  model.load_state_dict(torch.load("quote_model_epoch3.pt"))

if TRAIN:
  print(100*'-')
  print("Starting Training...")
  device = torch.device("cuda")

  LEARNING_RATE = 2e-5
  WARMUP_STEPS = 200
  EPOCHS = 4
  BATCH_SIZE = 16

  model = model.to(device)
  model.train()
  optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)

  train_dataloader = DataLoader(quotes, batch_size=1, shuffle=True)
  loss = 0
  batch_count = 0
  input_tensor = None
  
  for epoch in range(EPOCHS):
      print("Epoch: ", epoch)
      print("Loss: ", loss)

      # for step, entry in enumerate(tqdm(train_dataloader)):
      #   print(entry)
      #   if torch.is_nan(entry):
      #     continue
        
      #   if step > 10:
      #     raise Exception("paused")

      for step, entry in enumerate(tqdm(train_dataloader)):
        (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)
        if carry_on and step != len(train_dataloader) - 1:
            continue
        input_tensor = input_tensor.to(device)
        outputs = model(input_tensor, labels=input_tensor)
        # outputs = model(input_tensor)
        loss = outputs[0]
        loss.backward()
        if batch_count % BATCH_SIZE == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()
        batch_count += 1
        input_tensor = None
      name = "quote_model_epoch" + str(epoch) + ".pt"
      torch.save(model.state_dict(), name)

if LOAD:
  model = GPT2LMHeadModel.from_pretrained('gpt2')
  model.load_state_dict(torch.load("quote_model_epoch3.pt"))
  device =  torch.device("cuda")
  model.to(device)

----------------------------------------------------------------------------------------------------
Starting Training...
Epoch:  0
Loss:  0


100%|██████████| 1664/1664 [01:06<00:00, 25.09it/s]


Epoch:  1
Loss:  tensor(3.5138, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 1664/1664 [01:05<00:00, 25.25it/s]


Epoch:  2
Loss:  tensor(3.4108, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 1664/1664 [01:05<00:00, 25.28it/s]


Epoch:  3
Loss:  tensor(3.4517, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 1664/1664 [01:05<00:00, 25.40it/s]


In [20]:
generated_quotes

[' It\'s a very special day when you get to sit in this corner. And if I don\'t want to do that, the only way out is by doing it alone."\nThe next step for those who are interested can be getting married at age 35 or 40 and starting their own business as an artist after they have accomplished what has been done before them on stage -- something he did not believe would ever happen again (except maybe because of one woman).<|endoftext|>',
 ' The best example is when a man gets to the age of forty, and he goes off with an education. It\'s like having children: it gives you something new that people can grow up without going away altogether."<|endoftext|>',
 ' The soul is one person\'s spirit.To make a living, you must live in it yourself."<|endoftext|>',
 " If you love the world and care about what it does to us, then surely this will give a great deal of value.In addition I'd like some people who've already read that sorta thing from The Lord Of Poker or something...<|endoftext|>",
 " I